In [2]:
import os
import sys
import re
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import pickle
from scipy.optimize import fmin

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.append('../../statmechlib')
from statmechlib.read_write import read_vasp
from statmechlib.preprocessing import Trajectory, select_nodes, scale_configuration, pair_dist_cutoff
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM, get_stats_EAM_pairdist
from statmechlib.forcefields import sd2_loss, utot_EAM, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos

In [5]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [6]:
it = 4 # read in it (independent process)

In [7]:
n = int((1.45-0.85)/0.05 + 0.01)
reos = np.linspace(0.85, 1.45, n+1)*2.74

In [8]:
with open(os.path.join(working, 'stats_clean'+'.pickle'), 'rb') as fi:
    stats_ready = pickle.load(fi)

with open(os.path.join(working, 'target_clean'+'.pickle'), 'rb') as fi:
    targ_ready = pickle.load(fi)   

In [9]:
multi_index = []
multi_index.append([0, 3, 5, 8, 13, 17, 20, 24, 27, 31])
multi_index.append([2, 4, 8, 11, 13, 17, 20, 24, 27, 32])
multi_index.append([1, 4, 6, 9, 13, 17, 20, 24, 27, 31])
multi_index.append([1, 4, 7, 9, 13, 17, 20, 24, 27, 31])
multi_index.append([1, 3, 5, 8, 12, 16, 19, 23, 28, 31])
multi_index.append([0, 4, 6, 8, 12, 17, 20, 25, 28, 32])

In [10]:
stats_opts = []
for index in multi_index:
    index = np.array([True if i in index else False for i in range(len(stats_ready['hyperparams']))])
    print(np.array(stats_ready['hyperparams'])[index])
    stats_opts.append(select_nodes(stats_ready, index))

[2.4  2.65 2.72 2.75 3.   3.45 4.   4.5  5.   5.6 ]
[2.5  2.7  2.75 2.85 3.   3.45 4.   4.5  5.   5.75]
[2.45 2.7  2.73 2.77 3.   3.45 4.   4.5  5.   5.6 ]
[2.45 2.7  2.74 2.77 3.   3.45 4.   4.5  5.   5.6 ]
[2.45  2.65  2.72  2.75  2.9   3.3   3.75  4.356 5.15  5.6  ]
[2.4  2.7  2.73 2.75 2.9  3.45 4.   4.65 5.15 5.75]


In [11]:
#pars_dict['pair'] = [-28.301652628205268, -0.0022071786576090694, 0.027656591839734557, -6.543090393280296, 10.102407371395133, -2.478121982724172, 3.7048200337887582, -3.6346043391919514, 1.7181534576100437, -0.2520201994640249]
#pars_dict['embed'] = [-1.1825596913794016, 2.8944059727957714e-05]
pars_list = []
for index in multi_index:
    pars_dict = {}
    if len(np.array(stats_ready['hyperparams'])[index]) == 10:
        pars_dict['embed'] = [-6.84229349e-01,  1.52487644e-04]
        pars_dict['pair'] = [-6.56958142e+01, 0.0, 2.30194161e+01, 0.0, -3.48639263e+00,  3.45615281e+00, -8.88083580e-01, -1.34315802e+00, 1.24532843e+00, -3.51200974e-01]
    elif len(np.array(stats_ready['hyperparams'])[index]) == 9:
        pars_dict['embed'] = [-6.84229349e-01,  1.52487644e-04]
        pars_dict['pair'] = [-6.56958142e+01, 0.0, 2.30194161e+01, -3.48639263e+00,  3.45615281e+00, -8.88083580e-01, -1.34315802e+00, 1.24532843e+00, -3.51200974e-01]        
    elif len(np.array(stats_ready['hyperparams'])[index]) == 8:
        pars_dict['embed'] = [-6.84229349e-01,  1.52487644e-04]
        pars_dict['pair'] = [-6.56958142e+01, 2.30194161e+01, -3.48639263e+00,  3.45615281e+00, -8.88083580e-01, -1.34315802e+00, 1.24532843e+00, -3.51200974e-01]        
    else:
        raise ValueError('Unsupported number of parameters')
    print(len(np.array(stats_ready['hyperparams'])[index]), len(pars_dict['pair']))
    pars_list.append(pars_dict)

with open(os.path.join(working, "pars_in.pickle"), 'wb') as fo:
    pickle.dump(pars_list, fo)


10 10
10 10
10 10
10 10
10 10
10 10


In [12]:
# read corresponding parameters
with open(os.path.join(working, 'pars_in'+'.pickle'), 'rb') as fi:
    pars_list = pickle.load(fi)
print(pars_list)

[{'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}]


In [13]:
stats = []
target = []
for key in list(targ_ready.keys()):
    stats.append(stats_opts[it][key])
    target.append(targ_ready[key])

In [14]:
pars_dict = pars_list[it]
pars_in = pars_dict['embed'] + pars_dict['pair']
print(pars_in, len(pars_in))

[-0.684229349, 0.000152487644, -65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974] 12


In [15]:
target[0]['weight'] = 1.0
target[1]['weight'] = 2.0
target[2]['weight'] = 0.5
target[3]['weight'] = 1.0
target[4]['weight'] = 0.8

In [16]:
len(target[0]['energy']), [t['weight'] for t in target], target[1]['energy']

(1284,
 [1.0, 2.0, 0.5, 1.0, 0.8],
 [-1139.2,
  -906.9778906249999,
  -1127.13044453125,
  -1136.3561554687499,
  -1136.7059554687498,
  0.0,
  0.0])

In [17]:
def optimize_EAM(target, stats, ieam, multi_pars, n_b=3):
    
    optimal_parameters = []
    
    for i, pars in enumerate(multi_pars):
        sd_ini = sd2_loss(list(pars), stats, target, utot_EAM, None, [ieam])
        print(i,' sd2: ini', sd_ini, end='')
        output = fmin(sd2_loss, pars, args=(stats, target, utot_EAM, None, [ieam]), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
        optimal_parameters.append(tuple([output[1], output[0]]))
        print('sd2: final', output[1])

    n_best = min(n_b, len(optimal_parameters))
    #print(optimal_parameters)
                 
    best_params = sorted(optimal_parameters, key=lambda param: param[0])[:n_best]
    
    print('Best params:', best_params)

    return best_params

In [18]:
multi_pars = []

multi_pars.append(pars_in)

multi_pars.append([-1.48107607e+00, -2.11281591e-06, -7.75748552e+01,  1.23284902e+02,
       -1.05876828e+02,  2.56371829e+01,  7.21885424e-01, -2.01549825e+00,
        9.46930507e-01,  2.72867578e-01, -1.23624777e+00,  4.90853203e-01])

multi_pars.append([ 4.21813176e+00,  1.28438711e-02,  7.18447209e+02,  2.79873504e+01,
       -3.64063120e+01,  1.55668343e+01, -3.59671207e+00,  7.18079901e-02,
       -7.56657066e-02,  2.53180527e-01, -4.96952030e-01, -4.27379804e-02])

multi_pars.append([-1.70284536e+00,  1.77584742e-03, -2.03091714e+02,  6.67172987e+02,
       -1.39072153e+03,  7.95275451e+02, -2.44611978e+01,  5.09001525e+00,
       -3.57713253e-01, -1.04584266e+00,  1.30255105e+00, -4.09472026e-01])

multi_pars.append([ 1.49705729e+00,  8.14531512e-03, -8.15988329e+02,  5.87519886e+02,
       -1.01285873e+03,  5.10026120e+02, -1.11686150e+01,  1.51343754e+00,
        1.15612472e-01, -4.84858749e-01,  3.56870076e-01, -2.09915075e-01])

multi_pars.append([ 1.54864560e+00,  1.06062750e-02, -3.00996729e+02,  3.32437763e+02,
       -3.39714171e+02,  9.93126859e+01, -2.04873166e+01,  1.12932344e+00,
       -7.73962738e-02, -1.97622239e-01,  1.14699068e-01, -1.23093792e-01])

multi_pars.append([ 1.08153442e+00,  2.12340093e-03, -2.83828424e+02,  3.30196594e+02,
       -3.61260411e+02,  1.30278313e+02, -3.12845720e+01,  2.97822976e+00,
       -5.85372576e-02, -8.72679252e-01,  5.95310242e-01, -2.30722232e-01])

multi_pars.append([-1.84530425e+00, -2.18389467e-05, -1.46520729e+03,  1.16790147e+02,
       -1.41528209e+02,  7.82448301e+01,  4.04340173e+00, -2.96638562e+00,
        2.19448488e+00, -5.43928853e-01, -1.31154657e+00,  7.46848321e-01])


In [19]:
for ieam in range(6, len(stats_opts[it]['hyperparams'])):
    print('ieam:', ieam)
    m_pars = multi_pars
    for i in range(6):
        best_pars = optimize_EAM(target, stats, ieam, m_pars, n_b=len(m_pars))
        m_pars = [p[1] for p in best_pars]

ieam: 6
0  sd2: ini 6.653754807030036sd2: final 0.03775035459804807
1  sd2: ini 4.6783258418065135sd2: final 0.03868167378802309
2  sd2: ini 4.5944113129955415sd2: final 0.013060749376104555
3  sd2: ini 6.707148217696151sd2: final 0.011204239746450864
4  sd2: ini 3.18310748200587sd2: final 0.00922213495982782
5  sd2: ini 2.919451294768196sd2: final 0.009438637227939368
6  sd2: ini 5.508148051668595sd2: final 0.05309269099191579
7  sd2: ini 3.2983123976593625sd2: final 0.03855872335596242
Best params: [(0.00922213495982782, array([ 4.75867184e-01,  2.79378464e-02, -5.44642303e+02,  2.41121193e+02,
       -5.93130112e+02,  4.26086643e+02, -2.94931394e+01,  6.30141390e-01,
        6.57303421e-03, -9.96491648e-02,  3.01209636e-01, -2.40663530e-01])), (0.009438637227939368, array([ 5.73134669e-01,  3.22228151e-02, -2.81815577e+02,  9.21399853e+01,
       -2.54848533e+02,  1.88678369e+02, -1.81034987e+01, -6.87074185e-01,
       -1.95843222e-02,  4.21949422e-02,  1.94194546e-01, -2.00832834e

0  sd2: ini 0.008518747880285484sd2: final 0.008518747880285484
1  sd2: ini 0.008518747880285518sd2: final 0.008518747880285383
2  sd2: ini 0.008518747880285952sd2: final 0.008518747880285952
3  sd2: ini 0.008518747880286118sd2: final 0.008518747880286065
4  sd2: ini 0.008518747880334425sd2: final 0.008518747880307084
5  sd2: ini 0.008534352475932167sd2: final 0.008527155380304947
6  sd2: ini 0.008546829058183554sd2: final 0.008518747880290289
7  sd2: ini 0.008864073716160733sd2: final 0.00851874788029944
Best params: [(0.008518747880285383, array([ 2.25970481e-01,  2.90260998e-02, -4.10701301e+02,  1.47706832e+02,
       -3.63644528e+02,  2.61036603e+02, -2.00492509e+01, -8.63586604e-01,
        6.84027596e-01, -3.70743589e-01,  4.39408062e-01, -2.87835110e-01])), (0.008518747880285484, array([ 2.25970317e-01,  2.90260990e-02, -4.10701415e+02,  1.47706882e+02,
       -3.63644603e+02,  2.61036641e+02, -2.00492490e+01, -8.63586610e-01,
        6.84027572e-01, -3.70743586e-01,  4.3940808

0  sd2: ini 0.012621841313316184sd2: final 0.012617066893751694
1  sd2: ini 0.012664527791795657sd2: final 0.012649959790377311
2  sd2: ini 0.01277764531942722sd2: final 0.012622863738049692
3  sd2: ini 0.012995622556286154sd2: final 0.012976720862258411
4  sd2: ini 0.013987101872585042sd2: final 0.013126181966938575
5  sd2: ini 0.014253654234857794sd2: final 0.014147440045707328
6  sd2: ini 0.014435005377133488sd2: final 0.01289981022010938
7  sd2: ini 0.016211518803988134sd2: final 0.015369892569666005
Best params: [(0.012617066893751694, array([ 2.48736827e+00,  3.59141693e-03, -1.41596649e+03,  4.49310136e+02,
       -8.58117896e+02,  5.75915342e+02, -2.76365224e+01, -2.66232287e-02,
        1.13072342e+00, -6.68091351e-01,  3.09988626e-01, -3.26727024e-01])), (0.012622863738049692, array([ 2.60677357e+00,  3.67370671e-03, -1.42396162e+03,  4.44201681e+02,
       -8.40667783e+02,  5.62742457e+02, -2.70079140e+01, -6.22555732e-02,
        1.09259687e+00, -6.31951710e-01,  2.80237181

0  sd2: ini 6.324055988492171sd2: final 0.015511604131556625
1  sd2: ini 4.932755234066443sd2: final 0.01085921395795685
2  sd2: ini 7.229768535062863sd2: final 0.07041873709828796
3  sd2: ini 6.848503299497138sd2: final 0.02471691216596403
4  sd2: ini 7.226779780094186sd2: final 0.07550928822925898
5  sd2: ini 7.227983782928066sd2: final 0.1587305534094744
6  sd2: ini 7.206772263167009sd2: final 0.030575177040270744
7  sd2: ini 4.407342681454091sd2: final 0.010390245750074592
Best params: [(0.010390245750074592, array([-2.08977857e+00, -4.34873770e-05, -5.23986146e+02,  1.53423076e+02,
       -1.64375834e+02,  7.64776568e+01,  9.13358556e+00, -1.60777995e+00,
        9.21852610e-01, -3.58754332e-01, -4.16092427e-01,  3.92189630e-01])), (0.01085921395795685, array([-2.06936739e+00, -5.87725740e-06,  4.88440097e+01,  5.39632378e+01,
       -9.71263418e+01,  6.34838813e+01,  3.42289566e+00, -1.18450853e+00,
        9.24170696e-01, -5.20866712e-01, -3.23361635e-01,  3.54855162e-01])), (0.

0  sd2: ini 0.009402357395716094sd2: final 0.009402357395716094
1  sd2: ini 0.009402357395742396sd2: final 0.00940235739573229
2  sd2: ini 0.009402455912172824sd2: final 0.009402357566181675
3  sd2: ini 0.009403052365590211sd2: final 0.009403003348914876
4  sd2: ini 0.009414406864898046sd2: final 0.00940453267041145
5  sd2: ini 0.009525924171792578sd2: final 0.009406058163581785
6  sd2: ini 0.009597544811936493sd2: final 0.009488239062958174
7  sd2: ini 0.009750154129506645sd2: final 0.009668009551649474
Best params: [(0.009402357395716094, array([-1.94255100e+00, -9.52190731e-05, -7.77304420e+02,  3.29370188e+02,
       -5.85072093e+02,  3.74018846e+02, -6.10420849e+00, -6.08604011e-02,
        8.53146827e-01, -3.73603788e-01, -3.01427910e-01,  3.23115867e-01])), (0.00940235739573229, array([-1.94255009e+00, -9.52190921e-05, -7.77304805e+02,  3.29370234e+02,
       -5.85072317e+02,  3.74019071e+02, -6.10423775e+00, -6.08577786e-02,
        8.53147265e-01, -3.73603841e-01, -3.01427675e

../../statmechlib/statmechlib/forcefields/eam.py:226: RuntimeWarning: overflow encountered in exp
  eee = np.exp(-uuu)


sd2: final 12.268686118726542
3  sd2: ini 7.098768520721854sd2: final 0.07002064622640605
4  sd2: ini 7.229780862069606sd2: final 0.03646975082456318
5  sd2: ini 11.542857511745435sd2: final 11.542857511745435
6  sd2: ini 7.2295799164022085sd2: final 0.016737504600529276
7  sd2: ini 4.7186759836881365sd2: final 0.01076712739727512
Best params: [(0.008039386121510799, array([-1.02452290e+00,  2.33981881e-05, -4.03905463e+02,  1.53661785e+02,
       -2.21951470e+02,  1.28511699e+02,  2.79802238e+00, -1.36173067e+00,
        8.45857182e-01, -1.59846075e-01, -5.92812272e-01,  3.78462743e-01])), (0.01022617929971379, array([-9.39869326e-01,  3.99171348e-05, -2.04830017e+02, -3.57788147e-02,
        1.87020299e+01,  4.73654826e-02,  1.76340894e-02, -9.82522874e-01,
        7.94834694e-01, -2.48581231e-01, -5.04320702e-01,  3.23267260e-01])), (0.01076712739727512, array([-1.45707860e+00, -6.21936572e-05, -8.26149045e+02,  1.97959748e+02,
       -1.16131063e+02,  9.98227700e+00,  2.06798337e+0

0  sd2: ini 0.0077742305530438885sd2: final 0.007774230553036946
1  sd2: ini 0.0077742630148934675sd2: final 0.007774255658429296
2  sd2: ini 0.0077742713250195045sd2: final 0.007774230864127393
3  sd2: ini 0.00777433864129469sd2: final 0.007774327797620966
4  sd2: ini 0.007774801698590199sd2: final 0.007774285323720208
5  sd2: ini 0.008272032639118656sd2: final 0.007904149933736403
6  sd2: ini 11.542857511745435sd2: final 11.542857511745435
7  sd2: ini 12.268686118726542sd2: final 12.268686118726542
Best params: [(0.007774230553036946, array([-1.10651530e+00,  1.88282201e-06, -6.75213137e+02,  2.41833559e+02,
       -3.67542612e+02,  2.18700767e+02,  1.01277276e+00, -8.88928242e-01,
        6.62417261e-01,  3.00805846e-02, -7.26088829e-01,  4.49984268e-01])), (0.007774230864127393, array([-1.10635148e+00,  1.88592242e-06, -6.75454893e+02,  2.41966877e+02,
       -3.67852528e+02,  2.18917124e+02,  1.00188171e+00, -8.88096244e-01,
        6.62403026e-01,  2.99814428e-02, -7.25892700e-01

In [20]:
optimal_parameters = [(4.407570269501649e-05, [-1.21658318e+00,  3.24863580e-05, -4.73661687e+01,  2.29965951e+01,
       -3.42248343e+00,  3.73956980e+00, -1.39292737e+00, -8.33458565e-01,
        9.29769431e-01, -1.83568989e-01]), (6.17447812420396e-05, [-1.30551487e+00,  1.41920657e-05, -4.61160967e+01,  2.29352328e+01,
       -3.42360958e+00,  3.82313395e+00, -1.54715703e+00, -6.97925842e-01,
        8.68730239e-01, -1.54592379e-01]), (9.723561709685708e-05, [-1.34624502e+00,  7.49457487e-06, -4.58871674e+01,  2.24064873e+01,
       -3.46516902e+00,  4.02263760e+00, -1.89661680e+00, -4.31617423e-01,
        8.06406324e-01, -1.37562101e-01]), (0.00014123622250142564, [-1.34136558e+00,  1.01769934e-05, -4.93998032e+01,  2.21279115e+01,
       -3.50791272e+00,  4.16822104e+00, -2.13414308e+00, -2.66237999e-01,
        7.83819577e-01, -1.36703203e-01])]

In [21]:
optimal_parameters = [ieam9]

NameError: name 'ieam9' is not defined

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['relax']['xyz']])[:-2]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, target, utot_EAM, [9])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-4][:-2])
        print('Model:', opti_out[-4][:-2])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[-4][:-2]))/nat)
        plt.plot(np.array(targ_out[-4][:-2])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    print('Model:', opti_out[-4][:-2])

    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[-4][:-2]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_bcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, target, utot_EAM, [9])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-2][:])
        print('Model:', opti_out[-2][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[-2][:-1]))/nat)
        plt.plot(np.array(targ_out[-2][:-1])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot( np.array([o for o in opti_out[-2][:-1]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
len(targ_ready['md']['xyz'])

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['md']['xyz']])[:]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, target, utot_EAM,[9])
    #plt.title('Selected configurations at 0K')
    if i == 0:
        #print('Targ:', targ_out[0][:])
        #print('Model:', opti_out[0][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[0][:]))/nat)
        #plt.plot(np.array(targ_out[0][:])/nat, 'o', label='target'+str(i), markersize=10)
        plt.plot(np.array(targ_out[0][:]), 'o', label='target'+str(i), markersize=5)

        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 1:# and i != 12:
    #    continue
    print(i, params_uopt[0])

    #print('Opti:', [o for o in opti_out[-1][:-1]])
    #plt.plot(np.array([o for o in opti_out[0][:]])/nat, 'x--', label='optimized '+str(i))
    plt.plot(np.array([o for o in opti_out[0][:]]), '--', label='optimized '+str(i), lw=1.0)

#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]', fontsize=30)
plt.xlabel('configuation #', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.533)
#plt.ylim(-8.9, -8.35)

#plt.ylim(-1075,-1035)
#plt.ylim(-1000,-900)
#plt.ylim(-950,-900)
#plt.ylim(-1136,-1133)
plt.ylim(-1180,-800)
plt.xlim(0,820)
#plt.xlim(847,990)

#plt.ylim(-1180,-800)

plt.legend(fontsize=30)
plt.savefig(plt.savefig(os.path.join(working, 'bcc_300.png')))

In [ ]:
for i, params_uopt in enumerate(optimal_parameters):
    if params_uopt[0] > 1.1e-3:
        continue
    print(i, params_uopt[0], '\n', params_uopt[1])

In [ ]:
stats_opts[it]['hyperparams']

In [ ]:
pars_out = {}
pars_out['pair_nodes'] = stats_opts[it]['hyperparams']
pars_out['mb_node'] = 6
pars_out['params'] = [optimal_parameters[1]]

In [ ]:
optimal_list = []

#with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
#    optimal_list = pickle.load(fi)

#if type(optimal_list) != list:
#    optimal_list = [optimal_list]

optimal_list.append(pars_out)

with open(os.path.join(working, 'pars_out.pickle'), 'wb') as fo:
    pickle.dump(optimal_list, fo)

In [ ]:
with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
    optimal_list = pickle.load(fi)

In [ ]:
optimal_list

In [ ]:
# Total pair potential
pos = targ_ready['relax']['xyz'][0]
bx = targ_ready['relax']['box'][0]
rr, rx = pair_dist_cutoff(pos, bx, 5.5)

In [ ]:
# embedding function
F = lambda d, a: a[0]*d**0.5 + a[1]*d + a[2]*d**2

V = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

# Cohesive potential (cubic splines - same form as V)
phi = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

rho = lambda r, aa, kk: sum([a*(rk - r)**3 for a, rk in zip(aa, kk) if r < rk and r > 0.01])

In [ ]:
optimal_list[-1]['params'][0][1]

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
pair_y, many_y, V_x_r, rho_x_r

In [ ]:
ene = []
for a in np.linspace(0.5, 2.0, 101):
    z = a*rr
    u_pair = 0.5*np.sum([V(x, pair_y, V_x_r) for x in z.flatten() if x > 0.1])

    u_many = 0.0
    for i in range(z.shape[0]):
        d = np.sum([phi(x, rho_x_a, rho_x_r) for x in z[i] if x > 0.1])
        u_many += F(d, many_y)

    ene.append([a, u_pair, u_many, u_pair+u_many])

ene = np.array(ene)

In [ ]:
#plt.plot(ene[:,0], ene[:,1]/54, label='pair')
#plt.plot(ene[:,0], ene[:,2]/54, label='manybody')
plt.title('Equation of state (T=0K)')
plt.plot(ene[:,0]*2.74, [universal_eos(x, 'W') for x in ene[:,0]], label='universal EOS')
plt.plot(ene[:,0]*2.74, ene[:,3]/128, label='u-total')
plt.hlines(0.0,0.5,6.0,lw=1)
plt.legend()
plt.xlim(2.0, 5.75)
plt.xlim(2.6, 2.85)
plt.ylim(-10,10)
plt.ylim(-9.,-8.5)

plt.xlabel('x')
plt.ylabel(r'U')

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
V_x_r

In [ ]:
pair_a = optimal_list[-1]['params'][0][1][2:]#pars_in1[2:]
pair_r = optimal_list[-1]['pair_nodes']

rho_a = [1.0]
rho_r = [pair_r[optimal_list[-1]['mb_node']]]

par = optimal_list[-1]['params'][0][1][:2]
many_a = [par[0], 0.0, par[1]]

In [ ]:
# Ranges of densities and separations

nr = 5000
rcut = pair_r[-1]#stats_opt['hyperparams'][-1]
dr = rcut/nr

rhomax = 12.0*phi(1.5, rho_a, rho_r)
print('rhomax:', rhomax)
rhomax = 1000.0
nrho = 5000
drho = rhomax/nrho

ri = 1.0
ro = 2.0

# W properties
na = 74
ma = 183.84
lc = 3.1653
alat = 'bcc'

# System composition
n_el = 1
el_name = ['W']

In [ ]:
def u_trans(r, unucl, ueq, ri=1.0, ro=2.0):
    """Transition between the core and equillibrium potentials"""
    if r < ri:
        return unucl
    elif r < ro:
        x = (ro + ri - 2*r)/(ro - ri)
        eta = 3/16*x**5 - 5/8*x**3 + 15/16*x + 1/2
        return ueq + eta*(unucl - ueq)
    else:
        return ueq

In [ ]:
# prepare lists of values
dens = np.linspace(0., rhomax-drho, nrho, endpoint=True)
r = np.linspace(0., rcut-dr, nr, endpoint=True)
r2 = np.linspace(0., rcut, 201)

fembd = [F(d, many_a) for d in dens] # W
edens = np.array([phi(x, rho_a, rho_r) for x in r])

# 1. Cubic spline for r > r_o
f_outer = [V(x, pair_a, pair_r) for i, x in enumerate(r)]
# 2. Repulsive core for r < r_i (precalculate up to r_o)
f_inner = [0.0] + [u_core(x) for x in r[1:]]
# 3. Transition region for r_i < r < r_o
fpair = [x*u_trans(x, f_inner[i], f_outer[i]) for i, x in enumerate(r)]
#fpair = f_outer

In [ ]:
# LAMMPS format
pot_path = '../sim/potentials'

with open(os.path.join(pot_path, 'W_lv_3.eam.fs'), 'w') as fo:
    
    # Comment section
    fo.write('Comment 1\n')
    fo.write('Comment 2\n')
    fo.write('Comment 3\n')
    
    # Number of elements
    fo.write(f"{n_el:5d} ")
    for i in range(n_el):
        fo.write(f"{el_name[i]:2}")
    fo.write("\n")
    
    # number of grid points, step size, and cutoff
    fo.write(f"{nrho:5d}{drho:24.16e}{nr:5d}{dr:24.16e}{rcut:24.16e}\n")
    
    # atomic number, mass, lattice size, lattice type
    fo.write(f"{na:5d} {ma:14.4f} {lc:14.4f} {alat:10}\n")
    
    # Embeding function
    for i in range(nrho//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fembd[i*4:i*4+4]))
    
    # Electronic density
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*edens[i*4:i*4+4]))
        
    # Pair potential
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fpair[i*4:i*4+4]))